In [67]:
import pyaudio
import wave
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [69]:
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 5

p = pyaudio.PyAudio()

print("Recording...")
stream = p.open(format = FORMAT, channels = CHANNELS, rate = RATE, input = True, frames_per_buffer = CHUNK)

frames = []
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("Recording finished.")
stream.stop_stream()
stream.close()
p.terminate()


OUTPUT_FILENAME = "Audio.wav"
save_decision = input("Do you want to save the file? (YES/No): ")

if save_decision.lower() == 'yes':
    print("File will be saved!")
  
    with wave.open(OUTPUT_FILENAME, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
    print(f"File saved as {OUTPUT_FILENAME}")
else:
    print("File not saved.")

Recording...
Recording finished.


Do you want to save the file? (YES/No):  YES


File will be saved!
File saved as Audio.wav


In [71]:
print("Processing raw audio data...")
raw_audio = np.frombuffer(b''.join(frames), dtype=np.int16)
raw_audio = raw_audio.reshape(1, -1)  
print(f"Processed raw audio shape: {raw_audio.shape}")

Processing raw audio data...
Processed raw audio shape: (1, 79872)


In [73]:
print("Building the LSTM model...")
model = Sequential()

model.add(LSTM(128, input_shape=(raw_audio.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax')) 

Building the LSTM model...


In [75]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [77]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_8 (LSTM)                        │ (None, 79872, 128)          │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 79872, 128)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 207,050 (808.79 KB)

 Trainable params: 207,050 (808.79 KB)

 Non-trainable params: 0 (0.00 B)

In [79]:
#pip install SpeechRecognition

In [81]:
import speech_recognition as sr
recognizer = sr.Recognizer()
audio_file = "Audio.wav"
with sr.AudioFile(audio_file) as source:
    print("Processing Audio file...")
    audio_data = recognizer.record(source)
try:
    text = recognizer.recognize_google(audio_data, language="fa-IR")  
    print("Recognized Text: ")
    print(text)
except sr.UnknownValueError:
    print("Google Speech Recognition could not understand the Audio")
except sr.RequestError as e:
    print(f"Error with the Google Speech Recognition service: {e}")


Processing Audio file...
Recognized Text: 
‏hello world
